# COVID Global Clustering Scratchpad 

Do the same using Global data from John Hopkins University

https://github.com/CSSEGISandData/COVID-19   

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


## Clustering thoughts
- Take case + death counts
- Trending pattern (?  what is a good single value metric.. rolling 3 day or some momentum value?)
- Adding population data
- Population density ?
- Median age and age distributions

Attempt to cluster the countries ?


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
import pandas as pd
import requests

dfc_graw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
dfd_graw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

dfc_g = dfc_graw.drop(columns=['Lat','Long','Province/State']).groupby(['Country/Region']).sum().sort_values(dfc_graw.columns[-1], ascending=False)
dfc_g = dfc_g.transpose()  
dfc_g.index = pd.to_datetime(dfc_g.index)
display('cases',dfc_g.tail())
display('chg in cases',dfc_g.tail().diff(axis=0))

dfd_g = dfd_graw.drop(columns=['Lat','Long','Province/State']).groupby(['Country/Region']).sum().sort_values(dfd_graw.columns[-1], ascending=False)
dfd_g = dfd_g.transpose().tail(40)  # last 40 days ?
dfd_g.index = pd.to_datetime(dfd_g.index) 
display('deaths',dfd_g.tail())
display('chg in deaths',dfd_g.tail().diff(axis=0))

#display('details of', dfd_g['Sudan'].tail(15), dfc_g['Sudan'].tail(15))


'cases'

Country/Region,US,Spain,Italy,Germany,France,China,Iran,United Kingdom,Turkey,Switzerland,...,Sierra Leone,Belize,Bhutan,Gambia,Western Sahara,Malawi,Burundi,Papua New Guinea,South Sudan,Timor-Leste
2020-04-01,213372,104118,110574,77872,57749,82361,47593,29865,15679,17768,...,2,3,4,4,0,0,2,1,0,1
2020-04-02,243453,112065,115242,84794,59929,82432,50468,34173,18135,18827,...,2,3,5,4,0,3,3,1,0,1
2020-04-03,275586,119199,119827,91159,65202,82511,53183,38689,20921,19606,...,2,4,5,4,0,3,3,1,0,1
2020-04-04,308850,126168,124632,96092,90848,82543,55743,42477,23934,20505,...,4,4,5,4,0,4,3,1,0,1
2020-04-05,337072,131646,128948,100123,93773,82602,58226,48436,27069,21100,...,6,5,5,4,4,4,3,1,1,1


'chg in cases'

Country/Region,US,Spain,Italy,Germany,France,China,Iran,United Kingdom,Turkey,Switzerland,...,Sierra Leone,Belize,Bhutan,Gambia,Western Sahara,Malawi,Burundi,Papua New Guinea,South Sudan,Timor-Leste
2020-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-02,30081.0,7947.0,4668.0,6922.0,2180.0,71.0,2875.0,4308.0,2456.0,1059.0,...,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0
2020-04-03,32133.0,7134.0,4585.0,6365.0,5273.0,79.0,2715.0,4516.0,2786.0,779.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-04,33264.0,6969.0,4805.0,4933.0,25646.0,32.0,2560.0,3788.0,3013.0,899.0,...,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2020-04-05,28222.0,5478.0,4316.0,4031.0,2925.0,59.0,2483.0,5959.0,3135.0,595.0,...,2.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0


'deaths'

Country/Region,Italy,Spain,US,France,United Kingdom,Iran,China,Netherlands,Germany,Belgium,...,Maldives,Saint Lucia,Somalia,Guinea-Bissau,Malta,Guinea,Grenada,Fiji,Eswatini,South Sudan
2020-04-01,13155,9387,4757,4043,2357,3036,3316,1175,920,828,...,0,0,0,0,0,0,0,0,0,0
2020-04-02,13915,10348,5926,5398,2926,3160,3322,1341,1107,1011,...,0,0,0,0,0,0,0,0,0,0
2020-04-03,14681,11198,7087,6520,3611,3294,3326,1490,1275,1143,...,0,0,0,0,0,0,0,0,0,0
2020-04-04,15362,11947,8407,7574,4320,3452,3330,1656,1444,1283,...,0,0,0,0,0,0,0,0,0,0
2020-04-05,15887,12641,9619,8093,4943,3603,3333,1771,1584,1447,...,0,0,0,0,0,0,0,0,0,0


'chg in deaths'

Country/Region,Italy,Spain,US,France,United Kingdom,Iran,China,Netherlands,Germany,Belgium,...,Maldives,Saint Lucia,Somalia,Guinea-Bissau,Malta,Guinea,Grenada,Fiji,Eswatini,South Sudan
2020-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-02,760.0,961.0,1169.0,1355.0,569.0,124.0,6.0,166.0,187.0,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-03,766.0,850.0,1161.0,1122.0,685.0,134.0,4.0,149.0,168.0,132.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-04,681.0,749.0,1320.0,1054.0,709.0,158.0,4.0,166.0,169.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-05,525.0,694.0,1212.0,519.0,623.0,151.0,3.0,115.0,140.0,164.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Global Population Comparisons

In [45]:
# GLobal Population 
# https://github.com/datasets/population/blob/master/data/population.csv

df_gpop = pd.read_csv('https://raw.githubusercontent.com/datasets/population/master/data/population.csv')
df_gpop_2016 = df_gpop[df_gpop['Year'] == 2016]
df_gpop_2016 = df_gpop_2016.rename(columns={'Country Name': 'Country/Region'})

display(df_gpop_2016.head(), dfc_g.tail(1).transpose().count())  #262 countries / n countries
df_gmerged = pd.merge(left=dfc_g.tail(1).transpose(), right=df_gpop_2016, on='Country/Region').rename(columns={'Value':'Pop'})
df_gmerged = df_gmerged.rename(columns={ df_gmerged.columns[1]: "Cases" })

df_gmerged = pd.merge(left=dfd_g.tail(1).transpose(), right=df_gmerged, on='Country/Region').rename(columns={'Value':'Pop'})
df_gmerged = df_gmerged.rename(columns={ df_gmerged.columns[1]: "Deaths" })
       
display(df_gmerged.head())



,Country/Region,Country Code,Year,Value
56,Arab World,ARB,2016,4.064527e+08
113,Caribbean small states,CSS,2016,7.245472e+06
170,Central Europe and the Baltics,CEB,2016,1.029741e+08
227,Early-demographic dividend,EAR,2016,3.170542e+09
284,East Asia & Pacific,EAS,2016,2.296786e+09


2020-04-05    183
dtype: int64

,Country/Region,Deaths,Cases,Country Code,Year,Pop
0,Italy,15887,128948,ITA,2016,6.060059e+07
1,Spain,12641,131646,ESP,2016,4.644396e+07
2,France,8093,93773,FRA,2016,6.689611e+07
3,United Kingdom,4943,48436,GBR,2016,6.563724e+07
4,China,3333,82602,CHN,2016,1.378665e+09


In [46]:
# population density source
# http://api.worldbank.org/v2/en/indicator/EN.POP.DNST?downloadformat=csv   
from zipfile import ZipFile
import requests, zipfile, io

import urllib.request
urllib.request.urlretrieve('http://api.worldbank.org/v2/en/indicator/EN.POP.DNST?downloadformat=csv', "GDP.zip")
import zipfile
compressed_file = zipfile.ZipFile('GDP.zip') 
csv_file = compressed_file.open('API_EN.POP.DNST_DS2_en_csv_v2_887474.csv')  # pattern match how?
popdensity = pd.read_csv(csv_file, header=2)
popdensity = pd.concat([popdensity.iloc[:,0:2], popdensity['2018']], axis=1).rename(columns={'2018':'popdensity'})

df_gmerged2 = pd.merge(left=df_gmerged, right=popdensity, on='Country Code', how='left')

display(df_gmerged.shape, popdensity.shape, df_gmerged2.shape)
display(popdensity.head(), df_gmerged2.head())

(155, 6)

(264, 3)

(155, 8)

,Country Name,Country Code,popdensity
0,Aruba,ABW,588.027778
1,Afghanistan,AFG,56.937760
2,Angola,AGO,24.713052
3,Albania,ALB,104.612263
4,Andorra,AND,163.842553


,Country/Region,Deaths,Cases,Country Code,Year,Pop,Country Name,popdensity
0,Italy,15887,128948,ITA,2016,6.060059e+07,Italy,205.450748
1,Spain,12641,131646,ESP,2016,4.644396e+07,Spain,93.529058
2,France,8093,93773,FRA,2016,6.689611e+07,France,122.338396
3,United Kingdom,4943,48436,GBR,2016,6.563724e+07,United Kingdom,274.827392
4,China,3333,82602,CHN,2016,1.378665e+09,China,148.348833
